In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import mysql.connector  # mysql.connector 모듈을 임포트합니다.
from datetime import datetime  # 날짜 형식 변환을 위한 모듈

# 웹드라이버 초기화
driver = webdriver.Chrome()

# URL로 이동
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_09i.do#//HanaBank"
driver.get(url)

# 페이지 로딩 대기
time.sleep(10)  # 페이지 로딩 시간을 위한 대기. 필요에 따라 조정

# 통화 선택 드롭다운 메뉴 선택 (value 기반으로 선택, 예: 'EUR' 선택)
currency_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#curCd'))
currency_dropdown.select_by_value('CNY')  # 'EUR' 값에 해당하는 옵션 선택

# 조회 버튼 클릭
search_button = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
search_button.click()

# 선택 후 데이터가 갱신될 시간을 기다림
time.sleep(10)

# 날짜 추출 (nth-child(2)부터 nth-child(335)까지)
date_row = driver.find_element(By.CSS_SELECTOR, '#numLookHis2 > table > tbody > tr:nth-child(1)')
dates = date_row.find_elements(By.CSS_SELECTOR, 'th')

# Open, High, Low, Close 위치 추출 (nth-child(2)부터 nth-child(335)까지)
open_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr0 > td')
high_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr1 > td')
low_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr2 > td')
close_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr3 > td')

# 데이터를 저장할 리스트 초기화
data_list = []

# 날짜와 가격 데이터를 추출
for i in range(1, len(dates)):  # td:nth-child(2)부터 시작
    try:
        # 날짜 추출
        date_text = dates[i].text.strip()

        # 날짜 형식을 "YYYY-MM-DD"로 변환 (예시: 20230524 -> 2023-05-24)
        if len(date_text) == 8:  # 날짜 형식이 8자리일 경우
            date_formatted = datetime.strptime(date_text, '%Y%m%d').strftime('%Y-%m-%d')
        else:
            date_formatted = date_text  # 형식에 맞지 않으면 그대로 사용

        # 각 가격 데이터 추출
        open_price = open_row[i-1].text.strip().replace(',', '')
        high_price = high_row[i-1].text.strip().replace(',', '')
        low_price = low_row[i-1].text.strip().replace(',', '')
        close_price = close_row[i-1].text.strip().replace(',', '')

        # 데이터 저장
        data_list.append({
            'date': date_formatted,  # 변환된 날짜를 사용
            'open': open_price if open_price else 'N/A',
            'high': high_price if high_price else 'N/A',
            'low': low_price if low_price else 'N/A',
            'close': close_price if close_price else 'N/A',
            'state': 'CNY'
        })

    except Exception as e:
        print(f"Error processing data for index {i}: {e}")

# 데이터 출력 확인
for data in data_list:
    print(data)

# MySQL 연결 정보 설정 (MySQL에 데이터 삽입 부분 활성화 시 사용)
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}

# MySQL 연결
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# 데이터 삽입
for data in data_list:
    insert_query = """
    INSERT INTO hr_ex_table (date, open, high, low, close, state)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (
        data['date'],
        float(data['open']) if data['open'] != 'N/A' else None,  # 'N/A' 값 처리
        float(data['high']) if data['high'] != 'N/A' else None,
        float(data['low']) if data['low'] != 'N/A' else None,
        float(data['close']) if data['close'] != 'N/A' else None,
        data['state']
    ))
    print(data)

# 변경사항 커밋
conn.commit()

# 연결 종료
cursor.close()
conn.close()

# 웹드라이버 종료
driver.quit()